In [1]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

In [2]:
hand_landmarks_dataset = pd.read_csv(r'datasets/training_dataset_2.csv')

In [3]:
landmarks_train = hand_landmarks_dataset.iloc[:,:-1]
labels_train = hand_landmarks_dataset[['class']]
class_names = ['noGesture','ThumbsDown','ThumbsUp']





In [4]:
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(63,)),
    keras.layers.Dense(45,activation='relu'),
    keras.layers.Dense(45,activation='relu'),
    keras.layers.Dense(45,activation='relu'),
    keras.layers.Dense(3,activation='softmax')])


In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [7]:
model.fit(landmarks_train,labels_train,epochs=20)

Epoch 1/20
534/534 [==============================] - 1s 2ms/step - loss: 0.0129 - accuracy: 0.9960
Epoch 2/20
534/534 [==============================] - 1s 2ms/step - loss: 0.0190 - accuracy: 0.9945
Epoch 3/20
534/534 [==============================] - 1s 2ms/step - loss: 0.0125 - accuracy: 0.9965
Epoch 4/20
534/534 [==============================] - 1s 2ms/step - loss: 0.0166 - accuracy: 0.9951
Epoch 5/20
534/534 [==============================] - 1s 2ms/step - loss: 0.0129 - accuracy: 0.9960
Epoch 6/20
534/534 [==============================] - 1s 2ms/step - loss: 0.0110 - accuracy: 0.9971
Epoch 7/20
534/534 [==============================] - 1s 2ms/step - loss: 0.0194 - accuracy: 0.9943
Epoch 8/20
534/534 [==============================] - 1s 2ms/step - loss: 0.0128 - accuracy: 0.9964
Epoch 9/20
534/534 [==============================] - 1s 2ms/step - loss: 0.0093 - accuracy: 0.9971
Epoch 10/20
534/534 [==============================] - 1s 2ms/step - loss: 0.0159 - accuracy: 0.9953

In [15]:
test_loss, test_acc = model.evaluate(landmarks_test,labels_test,verbose=1)
print("Test accuracy:", test_acc)

NameError: name 'labels_test' is not defined

In [8]:
singlePt = np.array([0.6809579730033875,0.49881237745285034,-7.70261237903469e-07,0.6605132222175598,0.6004080176353455,-0.06278210133314133,0.60448157787323,0.7184981107711792,-0.10183024406433105,0.5570058226585388,0.8220229148864746,-0.13428549468517303,0.5229418277740479,0.9075675010681152,-0.15679648518562317,0.457375168800354,0.6299885511398315,-0.061119914054870605,0.4414730668067932,0.6307037472724915,-0.11047393828630447,0.5062484741210938,0.6073432564735413,-0.13001498579978943,0.5400584936141968,0.5940637588500977,-0.13805444538593292,0.44463562965393066,0.5381578803062439,-0.053154218941926956,0.46015694737434387,0.5287531614303589,-0.1082412526011467,0.5386949777603149,0.5295087099075317,-0.11956675350666046,0.5546990036964417,0.5400099158287048,-0.11866963654756546,0.45650842785835266,0.45076262950897217,-0.05317096784710884,0.4774942696094513,0.4430127739906311,-0.10615700483322144,0.5423054099082947,0.45961737632751465,-0.09656068682670593,0.5495580434799194,0.4753476679325104,-0.08081386238336563,0.4784812331199646,0.3710188865661621,-0.058526549488306046,0.4817431569099426,0.3614494204521179,-0.08917935937643051,0.5321279764175415,0.38465166091918945,-0.07898342609405518,0.5446741580963135,0.40512752532958984,-0.06498053669929504])


# Ensure the shape matches the input shape of your model
singlePt = singlePt.reshape(1, -1)  # Add the batch dimension

# Make predictions
predictions = model.predict(singlePt)

# Get the predicted class
predicted_class_index = np.argmax(predictions)
predicted_class = class_names[predicted_class_index]

print("Predicted class:", predicted_class)


1/1 [==============================] - 0s 156ms/step
Predicted class: ThumbsDown


In [9]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import pyautogui


# MediaPipe initialization
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Video capture initialization
cam = cv.VideoCapture(0)


# Main loop for capturing hand landmarks and making predictions
while True:
    success, frame = cam.read()
    if not success:
        print("No frame to show")
        continue
    
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    hands_detected = hands.process(frame_rgb)
    frame = cv.cvtColor(frame_rgb, cv.COLOR_RGB2BGR)
    
    if hands_detected.multi_hand_landmarks:
        for hand_landmarks in hands_detected.multi_hand_landmarks:
            # Extract landmarks
            landmark_row = []
            for landmark in hand_landmarks.landmark:
                landmark_row.extend([landmark.x, landmark.y, landmark.z])
            
            # Fill empty cells if less than 21 landmarks detected
            while len(landmark_row) < 63:
                landmark_row.extend([None, None, None])
            
            # Convert to numpy array
            hand_landmarks = np.array([landmark_row])
            
            # Predict using the trained model
            predictions = model.predict(hand_landmarks)
            predicted_class_index = np.argmax(predictions)
            predicted_gesture = class_names[predicted_class_index]
            
            # Print predictions on the screen
            org = (50, 50)  # Origin point for text
            font = cv.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 255, 255)  # White color
            thickness = 2
            lineType = cv.LINE_AA
        
            frame = cv.putText(frame, predicted_gesture, org, font, fontScale, color, thickness, lineType)
            print(predicted_gesture)
            
            # Define a dictionary mapping gesture indices to actions
            gesture_actions = {
                0: lambda: None,  # No action for case 0
                1: lambda: pyautogui.press('left'),
                2: lambda: pyautogui.press('right')  # Assuming pattern-3 corresponds to index 2
            }
            
            if predicted_class_index in gesture_actions:
                gesture_actions[predicted_class_index]()

            
    
    cv.imshow("Captures", frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cam.release()
cv.destroyAllWindows()


1/1 [==============================] - 0s 28ms/step
noGesture
1/1 [==============================] - 0s 37ms/step
ThumbsDown
1/1 [==============================] - 0s 28ms/step
ThumbsDown
1/1 [==============================] - 0s 28ms/step
ThumbsDown
1/1 [==============================] - 0s 46ms/step
ThumbsDown
1/1 [==============================] - 0s 129ms/step
noGesture
1/1 [==============================] - 0s 14ms/step
ThumbsUp
1/1 [==============================] - 0s 14ms/step
ThumbsUp
1/1 [==============================] - 0s 26ms/step
ThumbsUp
1/1 [==============================] - 0s 15ms/step
ThumbsUp
1/1 [==============================] - 0s 14ms/step
ThumbsUp
1/1 [==============================] - 0s 13ms/step
ThumbsDown
1/1 [==============================] - 0s 16ms/step
ThumbsDown
1/1 [==============================] - 0s 15ms/step
ThumbsDown
1/1 [==============================] - 0s 15ms/step
ThumbsDown
1/1 [==============================] - 0s 15ms/step
ThumbsDown
1/1